## Sheetal Mahajan_20MAI0066
## LSTM with Keras: 
We know that RNNs are capable of remembering the characteristics of previous inputs and outputs. But for how long can it remember. For certain cases, the immediate previous output may not just be enough to predict what’s coming and the network may have to rely on information from a further previous output. 

For example, consider the phrase “the green grass” and a sentence “I live in France and I can speak French”. To predict the bold word in the first phrase, RNN can rely on its immediate previous output of green, on the other hand, to predict “french”, the Network has to overlook an output that is further away. This is called long-term dependency. Unfortunately as that gap between the words grows, RNNs become unable to learn to connect the information.

Long Short Term Memory or LSTM networks are a special kind of RNNs that deals with the long term dependency problem effectively. LSTM networks have a repeating module that has 4 different neural network layers interacting to deal with the long term dependency problem.

### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file = "/content/drive/MyDrive/Colab Notebooks/datasets/GOOG.csv"
data = pd.read_csv(file)

In [5]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Open       252 non-null    float64
 2   High       252 non-null    float64
 3   Low        252 non-null    float64
 4   Close      252 non-null    float64
 5   Adj Close  252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB
None


In [7]:
data.head(5)

,Date,Close,Adj Close
Date,,,
2020-05-06,2020-05-06,1347.300049,1347.300049
2020-05-07,2020-05-07,1372.560059,1372.560059
2020-05-08,2020-05-08,1388.369995,1388.369995
2020-05-11,2020-05-11,1403.260010,1403.260010
2020-05-12,2020-05-12,1375.739990,1375.739990


### Preprocessing:

In [6]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_axis(data['Date'], inplace=True)
data.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [10]:
close_data = data['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.75
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = data['Date'][:split]
date_test = data['Date'][split:]

print("length of close_train: ", len(close_train))
print("length of close_test: ", len(close_test))

length of close_train:  189
length of close_test:  63


In [11]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

### Creating LSTM model:

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(10,activation='relu',input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
9/9 [==============================] - 2s 7ms/step - loss: 2594640.2750
Epoch 2/50
9/9 [==============================] - 0s 7ms/step - loss: 2578440.4750
Epoch 3/50
9/9 [==============================] - 0s 6ms/step - loss: 2627939.7000
Epoch 4/50
9/9 [==============================] - 0s 7ms/step - loss: 2861165.6250
Epoch 5/50
9/9 [==============================] - 0s 6ms/step - loss: 2680321.3750
Epoch 6/50
9/9 [==============================] - 0s 6ms/step - loss: 2662722.1750
Epoch 7/50
9/9 [==============================] - 0s 6ms/step - loss: 2472044.1750
Epoch 8/50
9/9 [==============================] - 0s 7ms/step - loss: 2488709.5500
Epoch 9/50
9/9 [==============================] - 0s 6ms/step - loss: 2682678.6250
Epoch 10/50
9/9 [==============================] - 0s 7ms/step - loss: 2687485.1000
Epoch 11/50
9/9 [==============================] - 0s 7ms/step - loss: 2483739.8000
Epoch 12/50
9/9 [==============================] - 0s 7ms/step - loss: 2588764.8500
E

### Prediction:

In [13]:
import plotly.graph_objs as go

prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



### Forecasting:

In [15]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = data['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 50
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [16]:
trace1 = go.Scatter(
    x = forecast,
    y = forecast_dates,
    mode = 'lines',
    name = 'Data'
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()